# ESSA É A VERSÃO 1
- USA OS DADOS DA PLANILHA: DADOSTCC_TESTE (FORA DO GIT)
- SIMULAÇÃO COM OS HUBS: 53 E 117, SPOKES = 19,20,781


## SIMULAÇÃO 1 -  CÓD ORIGINAL
- Simulação com hubs e spoke **SEM** restrição 8 e 9 e **COM** restrição 8 e 9
- Simulação **SEM** hub-hub considerado

## SIMULAÇÃO 2 - CENÁRIO 1
- Cenário já modificado com novas demandas somadas--- Demanda do hub 117 alocada (FORÇADA) na demanda do HUB 53
- Ler aba: CENARIO1
- Simulação com hubs e spoke **SEM** restrição 8 e 9 e **COM** restrição 8 e 9
- Simulação **SEM** hub-hub considerado

## SIMULAÇÃO 3 - CENÁRIO 2
- Cenário já modificado com novas demandas somadas--- Demanda do hub 117 alocada (FORÇADA) na demanda do HUB 53
- Retirada das linhas do spoke 117
- Ler aba: CENARIO2
- Simulação com hubs e spoke **SEM** restrição 8 e 9 e **COM** restrição 8 e 9
- Simulação **SEM** hub-hub considerado

## SIMULAÇÃO 4 - CENÁRIO 3
- Mesmos dados da **SIMULAÇÃO 1 (DADOS)**, mas utilizando o método **GANHO** para alterar a demanda --- Demanda do hub 117 alocada na demanda do HUB 53
- Simulação **SEM** hub-hub considerado
- Simulação com hubs e spoke **SEM** restrição 8 e 9 e **COM** restrição 8 e 9

## SIMULAÇÃO 5 - CENÁRIO 4
- Adição dos dados HUB - HUB manualmente: Rota 117-53 com DEMANDA E DISTANCIA estabelecido previamente no código
- **Sem** método **GANHO**
- **Sem** método **Hub-Hub**

## SIMULAÇÃO 6 - CENÁRIO 5
- Mesmos dados do Cenário 4
- Utilizando o método **GANHO** para alterar a demanda -- Demanda do Hub17 alocada para o Hub53
- Adição dos dados HUB - HUB manualmente: Rota 117-53 com DEMANDA E DISTANCIA estabelecido previamente no código


## SIMULAÇÃO 7 - CENÁRIO 6
- Mesmos dados do aba "DADOS"
- Utilizando o método **GANHO** : ATRIBUI DEMANDA ZERO PARA AS ROTAS NAO ESCOLHIDAS
- Simulação **SEM** hub-hub considerado

## SIMULAÇÃO 8 - CENÁRIO 7
- Adição dos dados HUB - HUB manualmente: Rota 117-53 com DEMANDA E DISTANCIA estabelecido previamente no código
- Utilizando o método **GANHO** : ATRIBUI DEMANDA ZERO PARA AS ROTAS NAO ESCOLHIDAS
- **Sem** método **Hub-Hub**


## SIMULAÇÃO 9 - CENÁRIO 8
- Mesmos dados do aba "DADOS"
- Criação de um novo DADOS1 com os valores HUB-HUB inseridos pelo código
- Utilizando o método **GANHO** : ATRIBUI DEMANDA ZERO PARA AS ROTAS NAO ESCOLHIDAS
- Simulação **COM** hub-hub considerado


In [726]:
from pulp import *
import pandas as pd
import numpy as np
from pandas import ExcelWriter

In [727]:
#Function para calcular o preço da passagem
def Tarifa(dist):
    return np.exp(-0.73953 * np.log(dist) + 4.44446)*dist

#Function para calcular o CASK
def Cask(dist, seats):
    return np.exp(1.728912 - 0.40453 * np.log(dist) - 0.35671 * np.log(seats))

In [728]:
#Leitura dos dados para construção da malha

DADOS1 = pd.read_excel('DadosTCC_TESTE.xlsx',sheet_name = 'Dados')
DADOS1.dropna(inplace=True)
DADOS1.reset_index(drop=True, inplace=True)

HUB = DADOS1['Hub'].astype(int)
HUBN = HUB.drop_duplicates()
numero_HUB = len(HUBN)
#HUB = HUB.values.tolist()

SPOKE = DADOS1['Spoke'].astype(int)
SPOKEN = SPOKE.drop_duplicates()
numero_SPOKE = len(SPOKEN)
#SPOKE = SPOKE.values.tolist()

DEMANDA = round(DADOS1['Demanda']).astype(int)
#DEMANDA = DEMANDA.values.tolist()

# DISTANCIA = round(DADOS1['Distancia']).astype(int)
# #DISTANCIA = DISTANCIA.values.tolist()

# # Definição do número de vertices (edges)
# N = len(DISTANCIA)

In [729]:
print(HUB)
print(HUBN)
print(SPOKE)
print(SPOKEN)
print(numero_SPOKE)
print(DEMANDA)
print(DISTANCIA)
print(N)


0     53
1     53
2    117
3    117
4     53
Name: Hub, dtype: int32
0     53
2    117
Name: Hub, dtype: int32
0     19
1     20
2     19
3     20
4    781
Name: Spoke, dtype: int32
0     19
1     20
4    781
Name: Spoke, dtype: int32
3
0    1738
1    2202
2      82
3     587
4      40
Name: Demanda, dtype: int32
0    1246
1     942
2    1466
3    1052
4    1345
5     212
6     212
Name: Distancia, dtype: int32
7


In [730]:
#Calculo do YIELD E CASK de HUB-HUB
#Método usando dictionary

DADOS5 = pd.read_excel(r'D:\GUSTAVO\UNB\TCC\UnB-tcc-gustavo-2021\Dados\Dados_TCC_2021.xlsx', sheet_name='Hubs')
Dist_hub ={}
tarifa_hub = {}
cask_hub ={}

for i in DADOS5['Hub Dist']:
#for i in HUBN:
    dic1 = {}
    dic2={}
    dic3 = {}
    #for j in range(len(DADOS5['Hub Dist'])):
    
    for j in range(len(DADOS5['Hub Dist'])):
        dic1[DADOS5['Hub Dist'][j]] = DADOS5[i][j]
        
        if i!= DADOS5['Hub Dist'][j]:
            dic2[DADOS5['Hub Dist'][j]] = Tarifa(DADOS5[i][j]) 
            
            dic4={}
            for anv in range(N_ANV):
                dic4[DADOS3['Aeronave'][anv]] = Cask(DADOS5[i][j], ASSENTOS[anv])
            dic3[DADOS5['Hub Dist'][j]] = dic4
            
    Dist_hub[i] = dic1
    tarifa_hub[i] = dic2
    cask_hub[i] = dic3


In [731]:
# Criando ligação HUB->HUB com soma de demanda

DATA = pd.DataFrame(columns=['Hub', 'Spoke','Demanda', 'Distancia'])

for spoke in SPOKEN:
    c = DADOS1['Hub'][DADOS1['Spoke']==spoke].index
    for index in c:
        if HUB[INDEX[spoke]] != HUB[index]:
            DATA2 = pd.DataFrame({"Hub":[HUB[index]],"Spoke":[HUB[INDEX[spoke]]],"Demanda":[DEMANDA[index]],"Distancia": [Dist_hub[HUB[index]][HUB[INDEX[spoke]]]]})
            DATA = DATA.append(DATA2, ignore_index=True)

DATA_HUB = pd.DataFrame(columns=['Hub', 'Spoke', 'Demanda','Distancia'])
lista_HH = DATA['Hub'].drop_duplicates()        
print('-----')
print(lista_HH)
print(DATA)
for hub in lista_HH:
    df1 = DATA[DATA['Hub']==hub]
    lista_spoke = df1['Spoke'].drop_duplicates().reset_index(drop=True)
    print('df1', df1)
    print('lista_spoke', lista_spoke)
    
    for j in lista_spoke:
        df2 = df1[df1['Spoke']==j]
        print('j=', j)
        print(df2)
        soma = 0
        for i in df2['Demanda']:
            soma += i 
        print('soma',soma)
        
        df3 = pd.DataFrame({'Hub':[hub], 'Spoke': [j], 'Demanda': [soma], 'Distancia': Dist_hub[hub][j]})
        print(df3)
        DATA_HUB = DATA_HUB.append(df3, ignore_index=True)
        
        print('---')
        print(DATA_HUB)
 


-----
0    117
Name: Hub, dtype: object
   Hub Spoke Demanda Distancia
0  117    53      82       212
1  117    53     587       212
df1    Hub Spoke Demanda Distancia
0  117    53      82       212
1  117    53     587       212
lista_spoke 0    53
Name: Spoke, dtype: object
j= 53
   Hub Spoke Demanda Distancia
0  117    53      82       212
1  117    53     587       212
soma 669
   Hub  Spoke  Demanda  Distancia
0  117     53      669        212
---
   Hub Spoke Demanda Distancia
0  117    53     669       212


In [732]:
# CENARIO 8
print(DADOS1)
# JUNÇÃO DE DADOS1 COM A NOVA TABELA
DADOS1 = DADOS1.append(DATA_HUB, ignore_index=True)
print(DADOS1)

HUB = DADOS1['Hub'].astype(int)
HUBN = HUB.drop_duplicates()
numero_HUB = len(HUBN)
#HUB = HUB.values.tolist()

SPOKE = DADOS1['Spoke'].astype(int)
SPOKEN = SPOKE.drop_duplicates()
numero_SPOKE = len(SPOKEN)
#SPOKE = SPOKE.values.tolist()

DEMANDA = DADOS1['Demanda'].astype(int)
#DEMANDA = DEMANDA.values.tolist()

DISTANCIA = DADOS1['Distancia'].astype(int)
#DISTANCIA = DISTANCIA.values.tolist()

# Definição do número de vertices (edges)
N = len(DISTANCIA)

   Hub  Spoke      Demanda    Distancia
0   53     19  1738.428245  1246.946553
1   53     20  2201.708699   942.714927
2  117     19    81.968214  1466.000000
3  117     20   586.852533  1052.000000
4   53    781    40.000000  1345.000000
   Hub Spoke  Demanda Distancia
0   53    19  1738.43   1246.95
1   53    20  2201.71   942.715
2  117    19  81.9682      1466
3  117    20  586.853      1052
4   53   781       40      1345
5  117    53      669       212


In [733]:
DEMANDA

0    1738
1    2201
2      81
3     586
4      40
5     669
Name: Demanda, dtype: int32

In [734]:
DADOS2 = pd.DataFrame(columns=['Passagem'])
for i in range(N):
    x = Tarifa(DISTANCIA[i])
    DADOS2 = DADOS2.append({'Passagem': x}, ignore_index=True)
DADOS2.reset_index(drop=True, inplace=True)
PASSAGEM = DADOS2['Passagem']
#PASSAGEM = PASSAGEM.values.tolist()

In [735]:
# Leitura dos dados sobre aeronaves
  
DADOS3 = pd.read_excel('DadosTCC_TESTE.xlsx',sheet_name = 'Aeronaves') 
DADOS3.dropna(inplace=True)
DADOS3.reset_index(drop=True, inplace=True)
DADOS3.drop_duplicates(inplace=True)

AERONAVE = DADOS3['Aeronave']
N_ANV = len(AERONAVE)

ASSENTOS = DADOS3['Assentos']
    
ALCANCE = DADOS3['Alcance']

In [736]:
# Cálculo do CASK por aeronave e rota

DADOS4 = pd.DataFrame(columns=['Cask'])
for i in range(N_ANV):
    for j in range(N):
        x = Cask(DISTANCIA[j], ASSENTOS[i])
        DADOS4 = DADOS4.append({'Cask': x}, ignore_index=True)
DADOS4.reset_index(drop=True, inplace=True)
CASK = DADOS4['Cask']
#CASK = CASK.values.tolist()

In [737]:
CASK

0     0.058944
1     0.066005
2     0.055192
3     0.063121
4     0.057149
5     0.120670
6     0.052157
7     0.058405
8     0.048837
9     0.055853
10    0.050569
11    0.106775
Name: Cask, dtype: float64

In [738]:
Ganho = PASSAGEM - DISTANCIA
print(Ganho)
#DATA = DADOS1.copy()

INDEX = {}
for spoke in SPOKEN:
    ref= -100000 #Valor de referencia o menor possível
    c = DADOS1['Hub'][DADOS1['Spoke']==spoke].index
    for index in c:
        if Ganho[index] >= ref:
            ref = Ganho[index] #novo valor de referencia 
            INDEX[spoke] = index
        #else:
            #DATA.iat[index,3] = 0.001 #Atribui um valor alto para DISTANCIA
            #DEMANDA[index] = 0 ## NAO DA CERTO - APAGAR
            #print('Não muda index')
    print('INDEX final do spoke',spoke, '-', INDEX[spoke])
            
    
     


0   -700.877588
1   -435.177961
2   -897.294579
3   -530.386416
4   -788.912995
5    131.672795
dtype: float64
INDEX final do spoke 19 - 0
INDEX final do spoke 20 - 1
INDEX final do spoke 781 - 4
INDEX final do spoke 53 - 5


In [740]:
NEW_DEMANDA = DEMANDA.copy()
#NEW_DEMANDA = DEMANDA
print(DEMANDA)
#print(NEW_DEMANDA, type(NEW_DEMANDA))

for spoke in SPOKEN:
    Inicial_hub = INDEX[spoke]
    c = DADOS1['Hub'][DADOS1['Spoke']==spoke].index
    demanda = 0
    for index in c:
        demanda += DEMANDA[index]
        if index != Inicial_hub:    # Nao pode repetir so a celula.... rodar o codigo inteiro (CELL-> RUN ALL ABOVE)
            #DEMANDA[index] = 0
            NEW_DEMANDA[index] = 0
    NEW_DEMANDA.iat[Inicial_hub] = demanda

print(DEMANDA)
print(NEW_DEMANDA)

DEMANDA = NEW_DEMANDA
print(DEMANDA)
#print(OLD_DEMANDA)


0    1738
1    2201
2      81
3     586
4      40
5     669
Name: Demanda, dtype: int32
0    1738
1    2201
2      81
3     586
4      40
5     669
Name: Demanda, dtype: int32
0    1819
1    2787
2       0
3       0
4      40
5     669
Name: Demanda, dtype: int32
0    1819
1    2787
2       0
3       0
4      40
5     669
Name: Demanda, dtype: int32


In [741]:
DEMANDA

0    1819
1    2787
2       0
3       0
4      40
5     669
Name: Demanda, dtype: int32

In [1440]:
## EXEMPLO PARA O DATA_HUB - UTILIZANDO CENARIO HIPOTETICO

# DF_exemplo = pd.DataFrame(data = {'HUB':[41,42,117,42,53,41,42,42], 
#                      'Spoke': [53,53,53,117,117,117,117,117], 
#                      'Demanda': [100,200,300,400,500,600,700,800],
#                       'Distancia': [1000,2000,3000,4000,5000,6000,7000,8000]})

# DF2_exemplo = pd.DataFrame(columns=['HUB', 'Spoke', 'Demanda','Distancia'])

# list_hub = DF_exemplo['HUB'].drop_duplicates()
# print('DF_exemplo:')
# print(DF_exemplo)
# print('list_hub:')
# print(list_hub)

# for hub in list_hub:
#     print('HUB=', hub)
#     DF2 = DF_exemplo[DF_exemplo['HUB']== hub]
#     list_spoke = DF2['Spoke'].drop_duplicates().reset_index(drop=True)
#     print('DF2:')
#     print(DF2)
#     print('list_spoke[HUB]:')
#     print(list_spoke)
    
#     for j in list_spoke:
#         print('J=', j)
#         DF3= DF2[DF2['Spoke']==j]
#         soma = 0
#         print('DF3')
#         print(DF3)
        
#         for i in DF3['Demanda']:
#             soma += i
#         DF4 = pd.DataFrame({'HUB':[hub], 'Spoke': [j], 'Demanda': [soma], 'Distancia': Dist_hub[hub][j]})
#         print('DF4')
#         print(DF4)
#         DF2_exemplo = DF2_exemplo.append(DF4, ignore_index=True)
#         print('DF2_exemplo')
#         print(DF2_exemplo)
        
    

In [639]:
# ROTAS = list(zip(HUB, SPOKE, DEMANDA, DISTANCIA, PASSAGEM))
# ROTAS

In [704]:
# DATA2= DATA
# for hub in HUBN:
#     for hub2 in HUBN:
#         if hub != hub2:
#             DATA3 = pd.DataFrame({"Hub":[hub],"Spoke":[hub2],"Demanda":[0],"Distancia": [Dist_hub[hub][hub2]]})
#             DATA2 = DATA2.append(DATA3, ignore_index=True)

In [754]:
M = 100
K = 2
    
FO = LpProblem('Malha_PAN',LpMaximize)

FLUXO = LpVariable.dicts('Fluxos',[(a,j) for j in range(N) for a in range(N_ANV)],0,None,cat='Integer')
PASSAGEIROS = LpVariable.dicts('PASSAGEIROS',[(a,j) for j in range(N) for a in range(N_ANV)],0,None,cat='Integer')
BINARIO1 = LpVariable.dicts('Bin1',[(a,j) for j in range(N) for a in range(N_ANV)],0,None,cat='Binary')
BINARIO2 = LpVariable.dicts('Bin2',[(a) for a in range(N_ANV)],0,None,cat='Binary')
BINARIO3 = LpVariable.dicts('Bin3',[(j,spoke,hubs) for j in range(N) for spoke in SPOKE for hubs in HUB],0,None, cat='Binary')

In [755]:
#Função Objetivo
FO += lpSum(PASSAGEM[j]*PASSAGEIROS[(a,j)] - CASK[((N*a)+j)]*ASSENTOS[a]*DISTANCIA[j]*FLUXO[(a,j)] for a in range(N_ANV) for j in range(N))

In [756]:
# R.1 - Num de pax transportado igual a demanda 
for j in range(N):
    FO += lpSum(PASSAGEIROS[(a,j)] for a in range(N_ANV)) - DEMANDA[j] <= 0

In [757]:
# R.2 - Demanda atendida pelo Fluxo
# Mesmo código do R.2 - Deixar apenas uma  
for a in range(N_ANV):
    for j in range(N):
        FO += PASSAGEIROS[(a,j)] - ASSENTOS[a]*FLUXO[(a,j)] <= 0

In [758]:
# R.2.2 - Demanda atendida pelo Fluxo
    #COD_ESTUDO_PAN: PAX - ASSENTO*FLUXO.
    # Mesmo código do R.2 - Deixar apenas uma 
# for j in range(N):
#     FO += lpSum(ASSENTOS[a]*FLUXO[(a,j)] for a in range(N_ANV)) - DEMANDA[j] >= 0

In [759]:
# R.3 - Definindo o Fluxo máximo
for a in range(N_ANV):
    for j in range(N):
        FO += FLUXO[(a,j)] - M*BINARIO1[(a,j)] <= 0

In [760]:
# R.4 - Número máximo de aeronave no modelo
for j in range(N):
    FO += lpSum(BINARIO1[(a,j)] for a in range(N_ANV)) - K <= 0

In [761]:
# R.5 - Alcance de aeronave 
for a in range(N_ANV):
    for j in range(N):
        FO += ALCANCE[a] - DISTANCIA[j]*BINARIO2[(a)] >= 0

In [762]:
# R.6 - Ajustando o Fluxo ao alcance
for a in range(N_ANV):
    for j in range(N):
        FO += FLUXO[(a,j)] - M*BINARIO2[(a)] <= 0


In [763]:
# R.7 - Num Max de anv por rota 
for j in range(N):
    FO += lpSum(BINARIO2[(a)] for a in range(N_ANV)) - K <= 0

In [764]:
# R.8 - Adequando o FLuxo à ligação Spike com apenas 1 HUB:

for spoke in SPOKEN:
    c = DADOS1['Hub'][DADOS1['Spoke']==spoke].index
    #print('SPOKE=',spoke)
    for j in c:
        #print('INDEX=',j)
        for a in range(N_ANV):
            FO += FLUXO[(a,j)] - M*BINARIO3[(j,SPOKE[j],HUB[j])] <= 0
            #print(FLUXO[(a,j)],'-M',BINARIO3[(j,SPOKE[j],HUB[j])])
            
        

In [765]:
# R.9 - Um SPIKE ligado por apenas 1 HUB
for spoke in SPOKEN:
    c = DADOS1['Hub'][DADOS1['Spoke']==spoke].index
    #print('SPOKE=',spoke)
    FO += lpSum(BINARIO3[(j,SPOKE[j],HUB[j])] for j in c) - 1 <=0
    #print(lpSum(BINARIO3[(j,SPOKE[j],HUB[j])] for j in c))
            

In [766]:
FO.solve()
print('Status:', LpStatus[FO.status])

Status: Optimal


In [767]:
print('\n')

for v in FO.variables():
    if v.varValue != 0 and v.varValue != M:
        print(v.name,'-',v.varValue)

lucro = value(FO.objective)

print('\n')  
print('O lucro total é de US$ %.2f' % lucro)



Bin1_(0,_0) - 1.0
Bin1_(0,_1) - 1.0
Bin1_(0,_2) - 1.0
Bin1_(0,_3) - 1.0
Bin1_(0,_4) - 1.0
Bin1_(0,_5) - 1.0
Bin1_(1,_0) - 1.0
Bin1_(1,_1) - 1.0
Bin1_(1,_2) - 1.0
Bin1_(1,_3) - 1.0
Bin1_(1,_4) - 1.0
Bin1_(1,_5) - 1.0
Bin2_0 - 1.0
Bin2_1 - 1.0
Bin3_(0,_19,_53) - 1.0
Bin3_(1,_20,_53) - 1.0
Bin3_(4,_781,_53) - 1.0
Bin3_(5,_53,_117) - 1.0
Fluxos_(0,_4) - 1.0
Fluxos_(0,_5) - 2.0
Fluxos_(1,_0) - 12.0
Fluxos_(1,_1) - 18.0
Fluxos_(1,_5) - 3.0
PASSAGEIROS_(0,_4) - 40.0
PASSAGEIROS_(0,_5) - 220.0
PASSAGEIROS_(1,_0) - 1819.0
PASSAGEIROS_(1,_1) - 2787.0
PASSAGEIROS_(1,_5) - 449.0


O lucro total é de US$ 2357265.61
